In [1]:
target_file = 'TEST/一首簡單的歌_主旋律.musicxml'
output_file = 'TEST/output/new_test_nega.xml'

import xml.etree.cElementTree as ET
tree = ET.parse(target_file)
root = tree.getroot()
root.tag, root.attrib
all_notes = root.findall('.//note')
any_notes = []

for note in all_notes:
    n={}
    if note.find('type') is not None:
        n['type'] = note.find('type').text

        if note.find('lyric') is not None:
            n['lyric'] = note.find('./lyric/text').text

        if note.find('pitch') is not None:
            n['pitch'] = note.find('./pitch/step').text + note.find('./pitch/octave').text
            if note.find('./pitch/alter') is not None:
                if note.find('./pitch/alter').text == '-1':
                    p = n['pitch'][0] + 'b' + n['pitch'][1]
                    n['pitch'] = p
                else:
                    n['pitch'] = n['pitch'][0] + '#' + n['pitch'][1]
        n['dot'] = 0
        if note.find('dot') is not None:
            n['dot'] = 1
        n['tie'] = []
        if len(note.findall('tie')) >0:
            tied_type = []
            for i in note.findall('tie'):
                tied_type.append(i.attrib['type'])
        
            n['tie'] = tied_type

    any_notes.append(n)    
    
pitch = {'C':0, 'D':2, 'E':4, 'F':5, 'G':7, 'A':9, 'B':11}
alter = {'#':1, 'b':-1}
duration = {'whole':1, 'half':1/2, 'quarter':1/4, 'eighth':1/8, '16th':1/16, '32nd':1/32}

note_list=[]

for i in any_notes:
    if 'type' not in i:
        continue
    
    note_pitch = 0
    dot = 0
    tie = 0
    #note_duration = duration[str(i['type'])]
    if 'pitch' in i:
        if len(i['pitch'])==2:
            note_pitch = pitch[str(i['pitch'][0])] + (int(i['pitch'][1])) *12

        else:
            note_pitch = pitch[str(i['pitch'][0])] + \
            alter[str(i['pitch'][1])] + (int(i['pitch'][2])) *12
    
    if 'dot' in  i:
        dot=i['dot']

    
    if 'tie' in i:
        tie=i['tie']

    
    note_list.append([i['type'], note_pitch, dot, tie])
    # C1=12
    
    

In [2]:
note_list

[['half', 0, 0, []],
 ['quarter', 0, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 50, 0, []],
 ['quarter', 53, 0, []],
 ['eighth', 53, 0, []],
 ['eighth', 53, 0, []],
 ['quarter', 55, 0, []],
 ['eighth', 53, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 57, 0, []],
 ['quarter', 55, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 50, 0, []],
 ['quarter', 53, 1, []],
 ['eighth', 53, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 58, 0, []],
 ['half', 60, 0, []],
 ['quarter', 0, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 60, 0, []],
 ['quarter', 62, 0, []],
 ['eighth', 62, 0, []],
 ['eighth', 60, 0, []],
 ['quarter', 57, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 60, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 53, 0, []],
 ['quarter', 50, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 50, 0, []],
 ['quarter', 53, 0, []],
 ['eighth', 5

In [3]:
pitch = {'C':0, 'D':2, 'E':4, 'F':5, 'G':7, 'A':9, 'B':11}
pitch_inver = {v: k for k, v in pitch.items()}
alter = {'#':1, 'b':-1}
duration_dict = {'whole':1, 'half':1/2, 'quarter':1/4, 'eighth':1/8,\
                 '16th':1/16, '32nd':1/32}
duration_dict_inver = {v: k for k, v in duration_dict.items()}


def creat_measure(number):
    measure_node = ET.Element('measure')
    measure_node.set('number',str(number))
    measure_attributes = ET.SubElement(measure_node, 'attributes')
    measure_attr_divi = ET.SubElement(measure_attributes, 'divisions')
    measure_attr_divi.text = '8'
    measure_attr_key = ET.SubElement(measure_attributes, 'key')
    measure_attr_key_fif = ET.SubElement(measure_attr_key, 'fifths')
    measure_attr_key_fif.text = '0'
    measure_attr_time = ET.SubElement(measure_attributes, 'time')
    measure_attr_time_beats = ET.SubElement(measure_attr_time, 'beats')
    measure_attr_time_beats.text = '4'
    measure_attr_time_beats_t = ET.SubElement(measure_attr_time, 'beat-type')
    measure_attr_time_beats_t.text = '4'
    measure_attr_clef = ET.SubElement(measure_attributes, 'clef')
    measure_attr_clef_sign = ET.SubElement(measure_attr_clef, 'sign')
    measure_attr_clef_sign.text = 'G'
    measure_attr_clef_line = ET.SubElement(measure_attr_clef, 'line')
    measure_attr_clef_line.text = '2'
    
    return measure_node

def creat_note(duration, pitch, if_dot, slur):
    note_node = ET.Element('note')
    if pitch == 0:
        rest = ET.SubElement(note_node, 'rest')
    else:
        alter = 0
        try :
            step = pitch_inver[int(pitch)%12]
        
        except KeyError:
            alter+=1
            pitch+=1
            step = pitch_inver[int(pitch)%12]
        
        octave = int(int(pitch)/12)
          
        note_pitch = ET.SubElement(note_node, 'pitch')
        note_pitch_step = ET.SubElement(note_pitch, 'step')
        note_pitch_step.text = str(step)
        note_pitch_octave = ET.SubElement(note_pitch, 'octave')
        note_pitch_octave.text = str(octave)
        if alter == 1:
            note_pitch_alter = ET.SubElement(note_pitch, 'alter')
            note_pitch_alter.text = str(-1)
            
    note_duration = ET.SubElement(note_node, 'duration')
    note_duration.text = str(int(duration_dict[str(duration)])*32)
    note_type = ET.SubElement(note_node, 'type')
    note_type.text = str(duration)
    
    if if_dot==1:
        note_dot = ET.SubElement(note_node, 'dot')
        
    if len(slur)==1:
        note_tie = ET.SubElement(note_node, 'tie')
        note_tie.set('type', slur[0])
        note_notations = ET.SubElement(note_node, 'notations')
        note_notations_tied = ET.SubElement(note_notations, 'tied')
        note_notations_tied.set('type', slur[0])
    
    elif len(slur)==2:
        note_tie = ET.SubElement(note_node, 'tie')
        note_tie.set('type', slur[0])
        note_tie = ET.SubElement(note_node, 'tie')
        note_tie.set('type', slur[1])
        note_notations = ET.SubElement(note_node, 'notations')
        note_notations_tied = ET.SubElement(note_notations, 'tied')
        note_notations_tied.set('type', slur[0])
        note_notations_tied = ET.SubElement(note_notations, 'tied')
        note_notations_tied.set('type', slur[1])

    return note_node
        
        
        
'''
def split_note(part_node, current_m, current_count, note_dur, note_pitch): #input part node
    first_gap = 1.0 - float(current_count)
    node_list = []
    if (first_gap) in duration_dict_inver:
        first_dur = duration_dict_inver[first_gap]
        first_note = creat_note(first_dur, note_pitch)
        
        current_m.append()
    else:
        ruler = 0.5
        while(True):
            if first_gap>ruler:
                break
            else:
                ruler = ruler/2.0
    
    '''
        

        

'\ndef split_note(part_node, current_m, current_count, note_dur, note_pitch): #input part node\n    first_gap = 1.0 - float(current_count)\n    node_list = []\n    if (first_gap) in duration_dict_inver:\n        first_dur = duration_dict_inver[first_gap]\n        first_note = creat_note(first_dur, note_pitch)\n        \n        current_m.append()\n    else:\n        ruler = 0.5\n        while(True):\n            if first_gap>ruler:\n                break\n            else:\n                ruler = ruler/2.0\n    \n    '

In [4]:
pitch = {'C':0, 'D':2, 'E':4, 'F':5, 'G':7, 'A':9, 'B':11} #deal with negative_harmony
negative_harmony ={0:7, 1:6, 2:5, 3:4, 4:3, 5:2, 6:1, 7:0, 8:11, 9:10, 10:9, 11:8,}

negative_list = []

for i in note_list:
    if i[1]==0:
        negative_list.append([i[0], 0, i[2], i[3]])
    else:
        number = negative_harmony[ i[1]%12 ] + int(i[1]/12)*12
        negative_list.append([i[0], number, i[2], i[3]])


In [5]:
import xml.etree.cElementTree as ET    #deal with the head

tree = ET.parse(target_file)
root = tree.getroot()
childs = root.getchildren()

for index, i in enumerate (childs):
    if i.tag=='part-list':
        new_part_list = childs[index]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """


In [6]:
new_part = ET.Element('part') # creat nodes
new_part.set('id', 'P1')

count_dur=0.0
count_num=1
current_m = creat_measure(count_num)
new_part.append(current_m)
for i in negative_list[:]:
    if count_dur>=1:
        print(count_dur)
        count_dur=0
        count_num+=1
        current_m = creat_measure(count_num)
        new_part.append(current_m)
        
        current_m.append(creat_note(i[0],i[1],i[2],i[3]))
        if i[2]==1: #note with dot
            count_dur+= (duration_dict[i[0]])*1.5
        else:
            count_dur+=duration_dict[i[0]]
    
    
    else:
        current_m.append(creat_note(i[0],i[1],i[2],i[3]))
        if i[2]==1: #note with dot
            count_dur+= (duration_dict[i[0]])*1.5
        else:
            count_dur+=duration_dict[i[0]]
    

    
    
    
    
    

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [7]:
negative_list

[['half', 0, 0, []],
 ['quarter', 0, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 53, 0, []],
 ['quarter', 50, 0, []],
 ['eighth', 50, 0, []],
 ['eighth', 50, 0, []],
 ['quarter', 48, 0, []],
 ['eighth', 50, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 58, 0, []],
 ['quarter', 48, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 53, 0, []],
 ['quarter', 50, 1, []],
 ['eighth', 50, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 57, 0, []],
 ['half', 67, 0, []],
 ['quarter', 0, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 67, 0, []],
 ['quarter', 65, 0, []],
 ['eighth', 65, 0, []],
 ['eighth', 67, 0, []],
 ['quarter', 58, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 67, 0, []],
 ['eighth', 57, 0, []],
 ['eighth', 58, 0, []],
 ['eighth', 48, 0, []],
 ['eighth', 50, 0, []],
 ['quarter', 53, 0, []],
 ['eighth', 55, 0, []],
 ['eighth', 53, 0, []],
 ['quarter', 50, 0, []],
 ['eighth', 5

In [8]:
'''new_part = ET.Element('part')
new_part.set('id', 'P1')

current_m = creat_measure(count_num)
new_part.append(current_m)
current_m.append(creat_note('eighth', 62))'''

"new_part = ET.Element('part')\nnew_part.set('id', 'P1')\n\ncurrent_m = creat_measure(count_num)\nnew_part.append(current_m)\ncurrent_m.append(creat_note('eighth', 62))"

In [9]:
new_root = ET.Element(root.tag)
new_root.set('version', '3.1')
new_root.append(new_part_list)


new_root.append(new_part)


In [10]:
def indent(elem, level=0):
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [11]:


    indent(new_root)
    tree = ET.ElementTree(new_root)
    tree.write(output_file)

In [12]:
def copy_Element( tree_root ):
    return ET.ElementTree( tree_root );    

n1 = ET.Element('sdsds')
n2 = ET.SubElement(n1, 'dot')
n3 = ET.SubElement(n1, 'dot')
n3.set('type','ss')
n3.attrib['type']='156'

n4 = copy_Element(n3)
nn = n1.findall('d123t')

print(n3.attrib['type'])

156


In [13]:
print(ET.tostring(n1,short_empty_elements=True)) 

b'<sdsds><dot /><dot type="156" /></sdsds>'


In [14]:
for i in n1:
    print(i)

<Element 'dot' at 0x000002BB9C840DB8>
<Element 'dot' at 0x000002BB9C840E08>
